In [1]:
import pandas as pd
import datetime as dt

#grab flight plan
flight_plan = pd.read_csv('iss-data/csv/iss_flight_plan_20220101-20251231.csv')

#query to only use dates where the event is dock and convert it to a list
dock_days_list = flight_plan[flight_plan['event'] == 'Dock']['datedim'].to_list()

#convert list of date strings into a list of datetime objects
dates_list = [dt.datetime.strptime(date, '%m/%d/%Y').date() for date in dock_days_list]

#print the days difference between each date
num_days = pd.Series(dates_list).diff().dt.days.iloc[1:].astype(int).tolist()
print("number of days til next dock\n",num_days)




number of days til next dock
 [4, 25, 22, 18, 24, 13, 43, 67, 15, 22, 12, 18, 76, 15, 5, 13, 67, 2, 13, 59, 21, 2, 19, 48, 29, 41, 10, 10, 17, 2, 15, 8, 30, 22, 7, 22, 29, 35, 16, 6, 14, 9, 44, 33, 4, 22, 1, 34, 3, 19, 17, 62, 4, 27, 48, 2, 29, 7, 25, 28, 25, 25]


In [2]:
#grab crew members per dock
crewF = pd.read_csv('iss-data/csv/iss_flight_plan_crew_20220101-20251321.csv')

crew_per_dock = []

for i, date in enumerate(dock_days_list):
    crewNum = crewF[crewF['datedim'] == date]['crew_count'].tolist()
    Amount = sum(crewNum)
    crew_per_dock.append(Amount)

#number of crew members on board during each dock
print("number of crew per docking\n",crew_per_dock)


number of crew per docking
 [7, 7, 10, 11, 11, 7, 7, 7, 10, 11, 7, 7, 7, 7, 7, 11, 7, 11, 11, 7, 7, 7, 11, 10, 7, 7, 11, 11, 7, 7, 11, 7, 10, 9, 7, 11, 7, 7, 11, 7, 7, 10, 10, 11, 7, 7, 7, 7, 7, 11, 7, 10, 11, 11, 7, 7, 11, 7, 10, 7, 7, 11, 7]


In [3]:
#grab rates
rater = pd.read_csv('iss-data/csv/rates_definition.csv')


#consumable must match a consumable on the rates csv. case sensitive.
consumable = 'ACY Inserts'
# consumable = 'Filter Inserts'
# consumable = 'KTO'
# consumable = 'Nitrogen'
# consumable = 'Oxygen'
# consumable = 'Urine Receptacle'
# consumable = 'US Food BOBs'
# consumable = 'Pretreat Tanks'
# consumable = 'Air'
# consumable = 'Water'


#sum of generated
sum_generated = sum(rater[(rater['affected_consumable'] == consumable) & (rater['type'] == 'generation')]['rate'].to_list())

#grabs the usage rates of consumable
usage_list = rater[(rater['affected_consumable'] == consumable) & (rater['type'] == 'usage')]['rate'].to_list()

#if units contains 'crew'
crew_check = rater[(rater['affected_consumable'] == consumable) & (rater['type'] == 'usage')]['units'].tolist()

sum_usage_list = []
sum_usage_crew_list = []

for i, j in enumerate(crew_check):
    if j.find('Crew') != -1:
        sum_usage_crew_list.append(usage_list[i])
    else:
        sum_usage_list.append(usage_list[i])



sum_usage = sum(sum_usage_list)
sum_usage_crew = sum(sum_usage_crew_list)

print(crew_check)
print(usage_list)
print('------------')
print(sum_usage_list)
print(sum_usage_crew_list)
print(sum_usage)
print(sum_usage_crew)





['Insert/Crew/Day']
[1.3]
------------
[]
[1.3]
0
1.3


In [7]:
import math

#create a list of base rates 
listofrates = [sum_usage] * (len(num_days))
listofrates_crew = []
consumable_to_send = []
current_sum = 0
res_list = []

for i in range(0, len(listofrates)):
    listofrates_crew.append(crew_per_dock[i] * sum_usage_crew)

for i in range(0, len(listofrates)):
    res_list.append(((listofrates[i] + listofrates_crew[i]) - sum_generated) * num_days[i])


#rounding
for i, value in enumerate(res_list):
    current_sum+=value
    consumable_to_send.append(math.trunc(current_sum))
    current_sum = current_sum - math.trunc(current_sum)
print(f"listofrates: {listofrates}")
print("consumable to send = num_days((sum_usage + (crew_per_dock * sum_usage_crew)) - sum_generated)")
print(consumable_to_send)
    


listofrates: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
consumable to send = num_days((sum_usage + (crew_per_dock * sum_usage_crew)) - sum_generated)
[36, 227, 286, 258, 343, 118, 392, 609, 195, 315, 109, 164, 692, 136, 46, 185, 610, 29, 186, 536, 192, 18, 271, 624, 264, 373, 143, 143, 155, 18, 215, 73, 390, 257, 64, 314, 264, 319, 229, 54, 128, 117, 572, 471, 37, 200, 9, 310, 27, 272, 154, 806, 57, 387, 436, 19, 414, 64, 325, 255, 227, 358]


In [5]:
##RUN ALL or this wont work
dock_days_list.pop()

readable = pd.DataFrame(
    {'Docking Days': dock_days_list,
     f'Amount of {consumable} to Send': consumable_to_send
    })

display(readable)

,Docking Days,Amount of ACY Inserts to Send
0,2/17/2022,36
1,2/21/2022,227
2,3/18/2022,286
3,4/9/2022,258
4,4/27/2022,343
...,...,...
57,9/3/2025,64
58,9/10/2025,325
59,10/5/2025,255
60,11/2/2025,227
